In [0]:
import numpy as np

##Функция

In [0]:
def nest_skok(x):
    f = 0.25 * (x[0] - 1) ** 2 + np.sum([(x1 - 2 * x0 ** 2 + 1) ** 2 for (x1, x0) in zip(x[1:], x[:-2])])
    return f

In [0]:
def grad_nest_skok(x):
    df = 0.5 * (x[0] - 1) + np.sum([(2 - 8 * x0) * (x1 - 2 * x0 ** 2 + 1) for (x1, x0) in zip(x[1:], x[:-2])])
    return df

In [0]:
def grad2_nest_skok(x):
    d2f = 0.5 + np.sum([(2 - 8 * (x1 - 6 * (x0 ** 2) + 1)) for (x1, x0) in zip(x[1:], x[:-2])])
    return d2f

In [4]:
x = [-1, 1, 1]
print('x:', x)
print('f(x):', nest_skok(x))
print('df(x):', grad_nest_skok(x))

x: [-1, 1, 1]
f(x): 1.0
df(x): -1.0


##Градиентный спуск

In [0]:
def gradient_descent(lr, x0, steps):
    x_prev = x0
    for i in range(steps):
        x = x_prev - lr * grad_nest_skok(x_prev)
        x_prev = x
    return x

In [15]:
lr = 1e-4
x0 = [-1, 1, 1, 1]
steps = 100
print('Before:', x0, nest_skok(x0))
x = gradient_descent(lr, x0, steps)
print('After:', x, nest_skok(x))

Before: [-1, 1, 1, 1] 1.0
After: [-0.99273903  1.00726097  1.00726097  1.00726097] 0.9945417091479689


##Стандартный метод Гаусса-Ньютона

In [0]:
def gn_method(x0, steps):
    x_prev = x0
    for i in range(steps):
        x = x_prev - (grad2_nest_skok(x_prev)) ** (-1) * grad_nest_skok(x_prev)
        x_prev = x
    return x

In [17]:
x0 = [-1, 1, 1, 1]
steps = 100
print('Before:', x0, nest_skok(x0))
x = gn_method(x0, steps)
print('After:', x, nest_skok(x))

Before: [-1, 1, 1, 1] 1.0
After: [-0.98532751  1.01467249  1.01467249  1.01467249] 0.9926760182169204


##Модифицированный метод Гаусса-Ньютона

In [0]:
def phi(x):
    return np.sum(x ** 2)


def psi(x, y):
    return phi(nest_skok(x) - grad_nest_skok(x) * (np.array(y) - np.array(x)))


def vmk(x, ys, M):
    vals = []
    for y in ys:
        buf = psi(x, y) - 0.5 * M * np.sum((np.array(y) - np.array(x)) ** 2)
        vals.append(buf)
    res = ys[np.argmin(vals)]
    return res, np.min(vals)

In [29]:
x = [-1, 1, 1]
ys = [[-2, 1, 1], [0, 1, 1]]
x_min, fm = vmk(x, ys, 1)
print(x_min, fm)

[-2, 1, 1] 1.5


##Метод трех квадратов

##Адаптивные варианты методов